In [1]:
import jax
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import compiling
import numpy as np
from tracr.compiler.validating import validate
from typing import Union, TypeVar
from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 
from rasp_generator import sampling, utils, map_primitives
import flax

from tracr.compiler.craft_model_to_transformer import NoTokensError
from tracr.compiler.basis_inference import InvalidValueSetError

rng = np.random.default_rng(0)

In [2]:
# generate dataset
from tracr.compiler.assemble import AssembledTransformerModel
from jaxtyping import ArrayLike


@flax.struct.dataclass
class RaspFlatDatapoint:
    """Ready for training"""
    program: ArrayLike
    weights: ArrayLike


@flax.struct.dataclass
class RaspTreeDatapoint:
    """Rasp program + tokenized rasp program + model weights + model instance"""
    program: rasp.SOp
    program_tokenized: list
    model: AssembledTransformerModel

In [3]:
from rasp_generator import sampling
from rasp_tokenizer import tokenizer


def sample():
    sampler = sampling.ProgramSampler(rng=rng, validate_compilation=True)
    sampler.sample()
    model, tokens, params = tokenizer.compile_and_tokenize(sampler.program)
    return model, tokens, params

In [4]:
for _ in range(1000):
    try:
        model, tokens, params = sample()
    except (InvalidValueSetError, NoTokensError) as err:
        print("Invalid program: {err}.")

Invalid program: {err}.


ValueError: Compiled program sequence_map_162 does not match RASP output.
Compiled output: [0, 4, 4, 9, 0]
RASP output: [0.0, 6.0, 6.0, 9, 0.0]
Test input: [3, 2, 1, 1, 0]
SOp: <tracr.rasp.rasp.SequenceMap object at 0x7f86f03c3e20>

In [ ]:
for _ in range(100):
    try:
        sampler = sampling.ProgramSampler(rng=rng)
        sampler.sample()
        model, tokens, params = tokenizer.compile_and_tokenize(sampler.program)
    except (InvalidValueSetError, NoTokensError) as err:
        print(f"Invalid program: {err}")

Invalid program
Invalid program
